<a href="https://colab.research.google.com/github/htsnet/sumarizacao-textos-processamento-linguagem-natural/blob/main/Udemy_Sumarizacao_com_similaridade_do_cosseno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sumarização com similaridade do cosseno

# Preparação do texto de exemplo

In [ ]:
import re
import nltk
import string
from nltk.cluster.util import cosine_distance
import networkx as nx
import numpy as np
from IPython.core.display import HTML

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')
print(stopwords)

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estiv

In [ ]:
def preprocessamento(texto):
  texto_formatado = texto.lower()
  tokens = []
  for token in nltk.word_tokenize(texto_formatado):
    tokens.append(token)

  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

  return texto_formatado

In [ ]:
texto_original = """
        A inteligência artificial é a inteligência similar à humana máquinas. Definem como
        o estudo de agente artificial com inteligência. Ciência e engenharia de
        produzir máquinas com inteligência. Resolver problemas e possuir
        inteligência. Relacionada ao comportamento inteligente. Construção de
        máquinas para raciocinar. Aprender com os erros e acertos. Inteligência
        artificial é raciocinar nas situações do cotidiano.
"""
texto_original = re.sub(r'\s+', ' ', texto_original)
texto_original

' A inteligência artificial é a inteligência similar à humana máquinas. Definem como o estudo de agente artificial com inteligência. Ciência e engenharia de produzir máquinas com inteligência. Resolver problemas e possuir inteligência. Relacionada ao comportamento inteligente. Construção de máquinas para raciocinar. Aprender com os erros e acertos. Inteligência artificial é raciocinar nas situações do cotidiano. '

Ver links no jupyter da aula (wikipedia e passo a passo)

# Função para calcular a similaridade entre as sentenças

In [ ]:
sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto_original)]
sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]

In [ ]:
sentencas_originais

[' A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [ ]:
sentencas_formatadas

['inteligência artificial inteligência similar humana máquinas',
 'definem estudo agente artificial inteligência',
 'ciência engenharia produzir máquinas inteligência',
 'resolver problemas possuir inteligência',
 'relacionada comportamento inteligente',
 'construção máquinas raciocinar',
 'aprender erros acertos',
 'inteligência artificial raciocinar situações cotidiano']

In [ ]:
def calcula_similaridade_sentencas(sentenca1, sentenca2):
  palavras1 = [palavra for palavra in nltk.word_tokenize(sentenca1)]
  palavras2 = [palavra for palavra in nltk.word_tokenize(sentenca2)]
  #print(palavras1)
  #print(palavras2)

  todas_palavras = list(set(palavras1 + palavras2)) # o set evita que as palavras fiquem duplicadas
  #print(todas_palavras)

  vetor1 = [0] * len(todas_palavras)
  vetor2 = [0] * len(todas_palavras)
  #print(vetor1)
  #print(vetor2)

  for palavra in palavras1:
    vetor1[todas_palavras.index(palavra)] += 1
  for palavra in palavras2:
    vetor2[todas_palavras.index(palavra)] += 1    

  #print(vetor1)
  #print(vetor2)  

  return 1 - cosine_distance(vetor1, vetor2) # quanto menor o número maior é a similaridade

In [ ]:
calcula_similaridade_sentencas(sentencas_formatadas[0], sentencas_formatadas[1]) # o resultado aqui é em % ==> 0.47 = 47%

0.4743416490252569

# Função para gerar matriz de similaridade

In [ ]:
def calcula_matriz_similaridade(sentencas):
  matriz_similaridade = np.zeros((len(sentencas), len(sentencas)))
  #print(matriz_similaridade)
  for i in range(len(sentencas)):
    for j in range(len(sentencas)):
      if i == j:
        continue
      matriz_similaridade[i][j] = calcula_similaridade_sentencas(sentencas[i], sentencas[j])

  return matriz_similaridade

In [ ]:
calcula_matriz_similaridade(sentencas_formatadas)

array([[0.        , 0.47434165, 0.47434165, 0.35355339, 0.        ,
        0.20412415, 0.        , 0.47434165],
       [0.47434165, 0.        , 0.2       , 0.2236068 , 0.        ,
        0.        , 0.        , 0.4       ],
       [0.47434165, 0.2       , 0.        , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.2       ],
       [0.35355339, 0.2236068 , 0.2236068 , 0.        , 0.        ,
        0.        , 0.        , 0.2236068 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.20412415, 0.        , 0.25819889, 0.        , 0.        ,
        0.        , 0.        , 0.25819889],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.47434165, 0.4       , 0.2       , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.        ]])

# Função para sumarizar

In [ ]:
def sumarizar(texto, quantidade_sentencas):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]
  matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)
  #print(matriz_similaridade)
  grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
  #print(grafo_similaridade.nodes)
  #print(grafo_similaridade.edges)
  notas = nx.pagerank(grafo_similaridade)
  #print(notas)
  notas_ordenadas = sorted(((notas[i], nota) for i, nota in enumerate(sentencas_originais)), reverse=True)
  #print(notas_ordenadas)
  melhores_sentencas = []
  for i in range(quantidade_sentencas):
    melhores_sentencas.append(notas_ordenadas[i][1])
  return sentencas_originais, melhores_sentencas, notas_ordenadas

In [ ]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(texto_original, 3)

In [ ]:
sentencas_originais

[' A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [ ]:
melhores_sentencas

[' A inteligência artificial é a inteligência similar à humana máquinas.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.',
 'Ciência e engenharia de produzir máquinas com inteligência.']

In [ ]:
notas_sentencas

[(0.22820924040178,
  ' A inteligência artificial é a inteligência similar à humana máquinas.'),
 (0.1839190802151221,
  'Inteligência artificial é raciocinar nas situações do cotidiano.'),
 (0.1633191450783496,
  'Ciência e engenharia de produzir máquinas com inteligência.'),
 (0.15437170333277758,
  'Definem como o estudo de agente artificial com inteligência.'),
 (0.12639273963873285, 'Resolver problemas e possuir inteligência.'),
 (0.0961690356396478, 'Construção de máquinas para raciocinar.'),
 (0.023809527846794958, 'Relacionada ao comportamento inteligente.'),
 (0.023809527846794958, 'Aprender com os erros e acertos.')]

# Visualização do resumo

In [ ]:
def visualiza_resumo(titulo, lista_sentencas, melhores_sentencas):
  texto = ''
  display(HTML(f'<h1>Resumo do texto - {titulo}</h1>'))
  for sentenca in lista_sentencas:
    #texto += sentenca
    if sentenca in melhores_sentencas:
      texto += str(sentenca).replace(sentenca, f"<mark>{sentenca}</mark>") + " "
    else:
      texto += sentenca + " "
  display(HTML(f"""{texto}"""))

In [ ]:
visualiza_resumo('Teste', sentencas_originais, melhores_sentencas)

# Extração de texto da internet

In [ ]:
!pip install goose3 # para extrair textos da internet

     |████████████████████████████████| 92kB 3.6MB/s 


In [ ]:
from goose3 import Goose

In [ ]:
g = Goose()
url = 'https://iaexpert.academy/2020/11/09/ia-preve-resultado-das-eleicoes-americanas/'
#url = 'https://cidadeemnumeros.com.br/2020/12/15/mp-instaura-inquerito-civil-sobre-escola-em-sao-caetano-do-sul/'
artigo = g.extract(url)

In [ ]:
artigo.cleaned_text

'Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton. Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos. Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados. Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.\n\nO modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais. O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito. No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.\n\nO Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano passado. Desta vez, o mode

In [ ]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(artigo.cleaned_text, 5)

In [ ]:
sentencas_originais

['Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton.',
 'Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos.',
 'Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados.',
 'Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.',
 'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.',
 'O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito.',
 'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.',
 'O Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano pas

In [ ]:
melhores_sentencas

['Desta vez, o modelo está treinando com cerca de 5 vezes mais dados que nas eleições americanas anteriores.',
 'Quando seu modelo foi usado para predizer os resultados da eleição corrente usando dados brutos, Joe Biden apareceu como vencedor com larga vantagem.',
 'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.',
 'Segundo o Dr. Makse, integrar estas duas variáveis em seu modelo é a parte mais importante do trabalho.',
 'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.']

In [ ]:
notas_sentencas

[(0.10099691917637997,
  'Desta vez, o modelo está treinando com cerca de 5 vezes mais dados que nas eleições americanas anteriores.'),
 (0.08211890797647801,
  'Quando seu modelo foi usado para predizer os resultados da eleição corrente usando dados brutos, Joe Biden apareceu como vencedor com larga vantagem.'),
 (0.07547555812522563,
  'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.'),
 (0.07334834938839217,
  'Segundo o Dr. Makse, integrar estas duas variáveis em seu modelo é a parte mais importante do trabalho.'),
 (0.07097153031645245,
  'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.'),
 (0.07084200739094762,
  'Parece que, desta vez, os algoritmos estão de fato contribuindo para que as predições sejam mais precisas.'),
 (0.06997085578620142,
  'A participação em redes sociais cos

In [ ]:
visualiza_resumo('Teste', sentencas_originais, melhores_sentencas)

# Solução para o exercício - lematização


In [ ]:
import spacy

In [ ]:
!python -m spacy download pt

     |████████████████████████████████| 21.2MB 1.0MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186283 sha256=90c235ddc26e8929da1a0a4cb9ea92153c6ea3554a381c6d4232dfda1e4729a6
  Stored in directory: /tmp/pip-ephem-wheel-cache-cxphupit/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [ ]:
pln = spacy.load('pt')
pln

In [ ]:
def preprocessamento_lematizacao(texto):
  texto = texto.lower()
  texto = re.sub(r" +", ' ', texto)

  documento = pln(texto)
  tokens = []
  for token in documento:
    tokens.append(token.lemma_)

  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

  return texto_formatado

In [ ]:
def sumarizar_lematizacao(texto, quantidade_sentencas):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [preprocessamento_lematizacao(sentenca_original) for sentenca_original in sentencas_originais]
  matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)
  grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
  notas = nx.pagerank(grafo_similaridade)
  notas_ordenadas = sorted(((notas[i], nota) for i, nota in enumerate(sentencas_originais)), reverse= True)
  melhores_sentencas = []
  for i in range(quantidade_sentencas):
    melhores_sentencas.append(notas_ordenadas[i][1])

  return sentencas_originais, melhores_sentencas, notas_ordenadas

In [ ]:
artigo.cleaned_text

'Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton. Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos. Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados. Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.\n\nO modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais. O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito. No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.\n\nO Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano passado. Desta vez, o mode

In [ ]:
sentencas_originais, melhores_sentencas, _ = sumarizar(artigo.cleaned_text, 5)
visualiza_resumo(artigo.title, sentencas_originais, melhores_sentencas)

In [ ]:
sentencas_originais, melhores_sentencas, _ = sumarizar_lematizacao(artigo.cleaned_text, 5)
visualiza_resumo(artigo.title, sentencas_originais, melhores_sentencas)